In [ ]:
# !pip install pandas transformers torch underthesea vncorenlp tqdm hf_xet


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json
import numpy as np
import pandas as pd
import torch
import math
from tqdm import tqdm
from underthesea import word_tokenize, pos_tag
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from sklearn.decomposition import TruncatedSVD

c:\Users\quang\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Perplexity là gì?

**Perplexity (PPL)** là một chỉ số đo độ “bối rối” của mô hình ngôn ngữ khi dự đoán từ tiếp theo trong câu.  

Nói cách khác: **mức độ khó dự đoán một chuỗi văn bản**.  
Công thức cơ bản cho một câu w_1, w_2, ..., w_N  với mô hình LM:

![alt text](image.png)

- *PPL >=1* 
- *PPL = 1* → câu quá dễ, mô hình dự đoán hoàn hảo (hiếm xảy ra).
- *PPL cao* → câu khó dự đoán, nhiều khả năng từ vựng khác nhau → độ phức tạp cao..


In [ ]:
def extract_linguistic_features(sentence, connectors, lit_terms):
    tokens = word_tokenize(sentence)
    sentence_length = len(tokens)
    avg_word_length = np.mean([len(t) for t in tokens]) if tokens else 0

    pos = pos_tag(sentence)
    num_nouns = sum(1 for _, p in pos if p.startswith("N"))
    num_verbs = sum(1 for _, p in pos if p.startswith("V"))
    num_adjs = sum(1 for _, p in pos if p.startswith("A"))
    num_lit_terms = sum(1 for t in tokens if t in lit_terms)
    num_clauses = sum(1 for t in tokens if t.lower() in connectors)
    num_punct = sum(1 for t in tokens if t in ".,;!?")

    return [sentence_length, avg_word_length, num_clauses, num_punct,
            num_nouns, num_verbs, num_adjs, num_lit_terms]

In [ ]:
def get_embedding(sentence,phobert_tokenizer,phobert_model,device, max_length=256):
    inputs = phobert_tokenizer(sentence, return_tensors="pt", truncation=True,
                               padding=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = phobert_model(**inputs)
    emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return emb

### Feature đánh giá độ phức tạp câu

##### Feature thống kê cú pháp (syntactic/structural features)

| Feature | Kiểu | Mô tả | Ví dụ |
|---------|------|-------|-------|
| sentence_length | Numeric | Số từ trong câu | 13 |
| avg_word_length | Numeric | Độ dài trung bình của từ | 3.3 |
| num_clauses | Numeric | Số mệnh đề ước lượng bằng liên từ | 0 |
| num_punct | Numeric | Số dấu câu | 0 |
| num_nouns | Numeric | Số danh từ (`N*`) | 8 |
| num_verbs | Numeric | Số động từ (`V*`) | 2 |
| num_adjs | Numeric | Số tính từ (`A*`) | 0 |


In [ ]:
def compute_perplexity(sentence,gpt2_tokenizer, gpt2_model,device, max_length=256):
    encodings = gpt2_tokenizer(sentence, return_tensors="pt",
                               truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = gpt2_model(**encodings, labels=encodings["input_ids"])
    return torch.exp(outputs.loss).item()

In [ ]:
def run_sen_comlex(input_file, output_file, connectors,phobert_tokenizer, phobert_model
                   , gpt2_tokenizer, gpt2_model, device, lit_terms, svd):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    embs = []
    meta = []

    for item in tqdm(data, desc="Processing questions"):
        qid = item["id"]
        subject = item["subject"]
        question = item["question"]

        # Feature ngôn ngữ học
        ling_feats = extract_linguistic_features(question, connectors, lit_terms)

        # Embedding PhoBERT
        emb = get_embedding(question,phobert_tokenizer, phobert_model,device)
        
        # Perplexity GPT2
        ppl = compute_perplexity(question, gpt2_tokenizer, gpt2_model, device)

        meta.append([qid, subject, question] + ling_feats + [ppl])
        embs.append(emb)

    embs = np.array(embs)
    reduced_embs = svd.fit_transform(embs)
    for i in range(len(data)):
        rows.append(meta[i] + reduced_embs[i].tolist())
    # ======================
    # Xuất CSV
    # ======================
    emb_dim = len(rows[0]) - 3 - 8 - 1  # trừ id, subject, question, 7 feature ngôn ngữ, ppl
    columns = ["id", "subject", "question",
            "sentence_length", "avg_word_length", "num_clauses", "num_punct",
            "num_nouns", "num_verbs", "num_adjs","num_lit_terms", "perplexity"] + [f"emb_{i}" for i in range(emb_dim)]

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(output_file, index=False, encoding="utf-8-sig")

    print("✅ Đã xuất độ phức tạp của câu hỏi sang file CSV:", output_file)
    print(df.head())

Processing questions: 100%|██████████| 2135/2135 [04:31<00:00,  7.85it/s]


✅ Đã xuất CSV: D:\NCKH\estimate_question_difficulty\Embedding\Output_features\question_features.csv
   id subject                                           question  \
0   1     văn  Từ Văn hóa trong văn bản Phong cách Hồ Chí Min...   
1   2     văn  Ý nào nói lên việc tiếp thu có chọn lọc tinh h...   
2   3     văn  Vấn đề chủ yếu được nói tới trong văn bản Phon...   
3   4     văn  Theo tác giả quan niệm thẩm mĩ về cuộc sống củ...   
4   5     văn         Từ nào sau đây trái nghĩa với truân chuyên   

   sentence_length  avg_word_length  num_clauses  num_punct  num_nouns  \
0                8         6.250000            0          0          4   
1               19         5.000000            0          0          7   
2               11         5.454545            0          0          4   
3               11         5.818182            0          0          5   
4                8         4.375000            1          0          2   

   num_verbs  num_adjs  ...   emb_758   emb_75